In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [43]:
torch.manual_seed(1) # 항상 똑같은 대괄(?)를 보장하기 위해

In [44]:
# Softmax
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0)  # max값을 확률로 나타내준다. 1의 경우 e^1 / e^1 + e^2 + e^3 = 0.09
print(hypothesis)

hypothesis.sum()

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [45]:
# Cross Entropy Loss ( Low-level ) 손실 함수

z = torch.rand(3, 5, requires_grad=True) # z = 3*5

# 예측
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

#정답
y = torch.randint(5, (3,)).long()
print(y)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)
tensor([0, 2, 1])


In [46]:
y_one_hot = torch.zeros_like(hypothesis) # = 3*5
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim=1에 대해서 y의 squeeze는 3 unsqueeze는 3*1 [0, 2, 1] => [0
                                        #                                                               2
                                        #                                                               1]

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [47]:
# y_one_hot * -torch.log(hypothesis))->(3*5)/sum(dim=1)->(3)/mean()->scalar
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [49]:
# Cross-entropy Loss with torch.nn.functional

# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [34]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [35]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [36]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y) #nll = Negative log likelihood

tensor(1.4689, grad_fn=<NllLossBackward>)

In [37]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [38]:
# Training with Low-level Cross Entropy Loss

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    hypothesis = F.softmax(x_train.matmul(w) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.411980
Epoch  100/1000 Cost: 0.344100
Epoch  200/1000 Cost: 0.299024
Epoch  300/1000 Cost: 0.271329
Epoch  400/1000 Cost: 0.254818
Epoch  500/1000 Cost: 0.244426
Epoch  600/1000 Cost: 0.237467
Epoch  700/1000 Cost: 0.232552
Epoch  800/1000 Cost: 0.228926
Epoch  900/1000 Cost: 0.226156
Epoch 1000/1000 Cost: 0.223978


In [39]:
# Training with F.cross_entropy

w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    z = x_train.matmul(w) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [40]:
# High-level Implementation with nn.Module

class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
      super().__init__()
      self.linear = nn.Linear(4, 3) # Output이 3

  def forward(self, x):
      return self.linear(x)

model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train) # x_train = m*4     prediction = m*3          y_train = m
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
